In [1]:
from opensearchpy import OpenSearch
import pandas as pd
import re
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import random
from PIL import Image
from scipy.sparse import bsr_matrix
from scipy import sparse
import json
from dotenv import load_dotenv
from tqdm import tqdm


# Set up connection

In [2]:
load_dotenv(dotenv_path="../server/.env")

OPENSEARCH_URL = os.getenv("OPENSEARCH_URL", "http://localhost:9200")
USERNAME = os.getenv("OPENSEARCH_USER", "admin")
PASSWORD = os.getenv("OPENSEARCH_PASS", "admin")

host = OPENSEARCH_URL.replace("https://", "").replace("http://", "").split(":")[0]
port = 443 if OPENSEARCH_URL.startswith("https") else 9200

client = OpenSearch(
    hosts=[{"host": host, "port": port}],
    http_auth=(USERNAME, PASSWORD),
    use_ssl=OPENSEARCH_URL.startswith("https"),
    verify_certs=False
)

print(f'running on {OPENSEARCH_URL}')

running on https://search-opensearch-cluster-wu5ju7kko4epwsbhzljedkegwm.aos.eu-north-1.on.aws


c:\Users\nerii\Desktop\Work\DVS\VORD\v1\_deployment\vord\preprocess\env_preprocess\lib\site-packages\opensearchpy\connection\http_urllib3.py:214: UserWarning: Connecting to https://search-opensearch-cluster-wu5ju7kko4epwsbhzljedkegwm.aos.eu-north-1.on.aws:443 using SSL with verify_certs=False is insecure.
  warnings.warn(


In [3]:
health = client.cluster.health()
print(health)

c:\Users\nerii\Desktop\Work\DVS\VORD\v1\_deployment\vord\preprocess\env_preprocess\lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search-opensearch-cluster-wu5ju7kko4epwsbhzljedkegwm.aos.eu-north-1.on.aws'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'cluster_name': '463470956574:opensearch-cluster', 'status': 'yellow', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'discovered_master': True, 'discovered_cluster_manager': True, 'active_primary_shards': 12, 'active_shards': 12, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 5, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 70.58823529411765}


In [8]:
index = "masks"
shards = client.cat.shards(index=index, format='json')

for shard in shards:
    print(shard)


{'index': 'masks', 'shard': '0', 'prirep': 'p', 'state': 'STARTED', 'docs': '587', 'store': '27.9mb', 'ip': 'x.x.x.x', 'node': '2e00c1c4aa5c970bdf3fd47587b8de4a'}
{'index': 'masks', 'shard': '0', 'prirep': 'r', 'state': 'UNASSIGNED', 'docs': None, 'store': None, 'ip': None, 'node': None}
{'index': 'masks', 'shard': '1', 'prirep': 'p', 'state': 'STARTED', 'docs': '603', 'store': '27.8mb', 'ip': 'x.x.x.x', 'node': '2e00c1c4aa5c970bdf3fd47587b8de4a'}
{'index': 'masks', 'shard': '1', 'prirep': 'r', 'state': 'UNASSIGNED', 'docs': None, 'store': None, 'ip': None, 'node': None}
{'index': 'masks', 'shard': '2', 'prirep': 'p', 'state': 'STARTED', 'docs': '610', 'store': '28.1mb', 'ip': 'x.x.x.x', 'node': '2e00c1c4aa5c970bdf3fd47587b8de4a'}
{'index': 'masks', 'shard': '2', 'prirep': 'r', 'state': 'UNASSIGNED', 'docs': None, 'store': None, 'ip': None, 'node': None}
{'index': 'masks', 'shard': '3', 'prirep': 'p', 'state': 'STARTED', 'docs': '578', 'store': '27.1mb', 'ip': 'x.x.x.x', 'node': '2e00c

c:\Users\nerii\Desktop\Work\DVS\VORD\v1\_deployment\vord\preprocess\env_preprocess\lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search-opensearch-cluster-wu5ju7kko4epwsbhzljedkegwm.aos.eu-north-1.on.aws'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [11]:
settings = client.indices.get_settings(index=index)
print(settings)

c:\Users\nerii\Desktop\Work\DVS\VORD\v1\_deployment\vord\preprocess\env_preprocess\lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search-opensearch-cluster-wu5ju7kko4epwsbhzljedkegwm.aos.eu-north-1.on.aws'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'masks': {'settings': {'index': {'replication': {'type': 'DOCUMENT'}, 'number_of_shards': '5', 'provided_name': 'masks', 'creation_date': '1744723292848', 'number_of_replicas': '1', 'uuid': 'mB4JdKSwRtWOUsOmr36EDQ', 'version': {'created': '136387827'}}}}}


# Set number of replicas to 0 (single node setup)

In [12]:
client.indices.put_settings(
    index='masks',
    body={
        "index": {
            "number_of_replicas": 0
        }
    }
)


c:\Users\nerii\Desktop\Work\DVS\VORD\v1\_deployment\vord\preprocess\env_preprocess\lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search-opensearch-cluster-wu5ju7kko4epwsbhzljedkegwm.aos.eu-north-1.on.aws'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'acknowledged': True}

In [13]:
print(client.cluster.health())

{'cluster_name': '463470956574:opensearch-cluster', 'status': 'green', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'discovered_master': True, 'discovered_cluster_manager': True, 'active_primary_shards': 12, 'active_shards': 12, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 0, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 100.0}


c:\Users\nerii\Desktop\Work\DVS\VORD\v1\_deployment\vord\preprocess\env_preprocess\lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search-opensearch-cluster-wu5ju7kko4epwsbhzljedkegwm.aos.eu-north-1.on.aws'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
